# PROJECT: Traffic Sign Recognition System
A complete end-to-end ML pipeline for traffic sign classification

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, auc, precision_recall_curve
)
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping,
    ReduceLROnPlateau, TensorBoard
)
from tensorflow.keras.applications import MobileNetV2
import cv2
import os
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))


TensorFlow Version: 2.20.0
GPU Available: []


Getting the data

## Download the dataset. Only need in google-colab

In [9]:
from google.colab import drive
drive.mount('/content/drive')

zip_path = "/content/drive/MyDrive/chinese-traffic-sign-dataset.zip"

!unzip "$zip_path" -d data/


In [11]:
import pandas as pd

annotations_path = "data/chinese-traffic-sign-dataset/annotations.csv"
df = pd.read_csv(annotations_path)

df.head()


,file_name,width,height,x1,y1,x2,y2,category
0,000_0001.png,134,128,19,7,120,117,0
1,000_0002.png,165,151,23,12,149,138,0
2,000_0003.png,128,122,22,14,116,105,0
3,000_0010.png,80,73,14,8,67,63,0
4,000_0011.png,186,174,36,15,155,157,0


In [13]:
import os
import pandas as pd

class DataAcquisition:
    """
    Handles loading and organizing the Chinese Traffic Sign dataset
    located in Google Colab after extraction.
    """

    def __init__(self, data_dir='data/'):
        self.data_dir = data_dir
        self.images_dir = os.path.join(data_dir, "images")
        self.annotations_path = os.path.join(data_dir, "annotations.csv")

        self.train_dir = os.path.join(data_dir, 'train')
        self.test_dir = os.path.join(data_dir, 'test')

    def setup_directories(self):
        os.makedirs(self.train_dir, exist_ok=True)
        os.makedirs(self.test_dir, exist_ok=True)
        print(f"✓ Created directories inside: {self.data_dir}")

    def load_annotations(self):
        if not os.path.exists(self.annotations_path):
            raise FileNotFoundError("annotations.csv not found. Make sure the dataset was extracted.")

        df = pd.read_csv(self.annotations_path)
        print(f"✓ Loaded annotations: {df.shape[0]} rows")
        self.annotations = df
        return df

    def load_dataset_info(self):
        """
        Automatically extract class IDs (and names if present)
        from the annotations CSV.
        """
        if not hasattr(self, 'annotations'):
            self.load_annotations()

        df = self.annotations

        # Get numeric class ids
        unique_classes = sorted(df['category'].unique())

        # Check if class names exist in the CSV
        if 'class' in df.columns or 'label' in df.columns:
            class_col = 'class' if 'class' in df.columns else 'label'
            mapping = df[['category', class_col]].drop_duplicates()
            class_names = mapping.set_index('category')[class_col].to_dict()
        else:
            # Fallback -> generic names
            class_names = {cid: f"class_{cid}" for cid in unique_classes}

        print(f"✓ Found {len(class_names)} classes")
        self.class_names = class_names
        return class_names


# Initialize data acquisition
data_acq = DataAcquisition(data_dir="data/chinese-traffic-sign-dataset/")
data_acq.setup_directories()

df = data_acq.load_annotations()


class_names = data_acq.load_dataset_info()

print(f"Dataset has {len(class_names)} traffic sign classes")


✓ Created directories inside: data/chinese-traffic-sign-dataset/
✓ Loaded annotations: 6164 rows
✓ Found 58 classes
Dataset has 58 traffic sign classes


# Data preprocessing

In [14]:
class DataPreprocessor:
    """
    Complete data preprocessing pipeline
    """

    def __init__(self, img_size=(64, 64), batch_size=32):
        self.img_size = img_size
        self.batch_size = batch_size

    def load_and_preprocess_image(self, image_path):
        """Load and preprocess a single image"""
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, self.img_size)
        img = img / 255.0  # Normalize to [0, 1]
        return img

    def create_augmentation_pipeline(self):
        """Create data augmentation pipeline"""
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.15,
            shear_range=0.1,
            brightness_range=[0.8, 1.2],
            horizontal_flip=False,  # Traffic signs shouldn't be flipped
            fill_mode='nearest'
        )
        return datagen

    def load_dataset(self, data_dir, split_ratio=0.2):
        """
        Load and split dataset

        Returns:
            X_train, X_val, X_test, y_train, y_val, y_test
        """
        images = []
        labels = []

        # Load images from directory structure
        for class_id in os.listdir(data_dir):
            class_path = os.path.join(data_dir, class_id)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_file)
                    try:
                        img = self.load_and_preprocess_image(img_path)
                        images.append(img)
                        labels.append(int(class_id))
                    except Exception as e:
                        print(f"Error loading {img_path}: {e}")

        X = np.array(images)
        y = np.array(labels)

        # Split into train, validation, and test
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=split_ratio*2, random_state=42, stratify=y
        )
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
        )

        print(f"\n📊 Dataset Split:")
        print(f"Training: {len(X_train)} images")
        print(f"Validation: {len(X_val)} images")
        print(f"Testing: {len(X_test)} images")

        return X_train, X_val, X_test, y_train, y_val, y_test

    def analyze_dataset(self, y_train, y_val, y_test):
        """Analyze dataset distribution"""
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))

        for idx, (y, title) in enumerate([
            (y_train, 'Training'),
            (y_val, 'Validation'),
            (y_test, 'Test')
        ]):
            unique, counts = np.unique(y, return_counts=True)
            axes[idx].bar(unique, counts, color='skyblue', edgecolor='navy')
            axes[idx].set_title(f'{title} Set Distribution')
            axes[idx].set_xlabel('Class ID')
            axes[idx].set_ylabel('Count')
            axes[idx].grid(axis='y', alpha=0.3)

        plt.tight_layout()
        plt.savefig('visualizations/dataset_distribution.png', dpi=300)
        print("\n✓ Saved dataset distribution visualization")

# Initialize preprocessor
preprocessor = DataPreprocessor(img_size=(64, 64), batch_size=32)

# Model Creation


In [15]:
class TrafficSignModel:
    """
    CNN model for traffic sign classification
    """

    def __init__(self, input_shape=(64, 64, 3), num_classes=58):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = None
        self.history = None

    def build_cnn_model(self):
        """Build custom CNN architecture"""
        model = models.Sequential([
            # Block 1
            layers.Conv2D(32, (3, 3), activation='relu',
                         input_shape=self.input_shape, padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Block 2
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Block 3
            layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.4),

            # Dense layers
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(self.num_classes, activation='softmax')
        ])

        self.model = model
        return model

    def build_transfer_learning_model(self):
        """Build model using transfer learning (MobileNetV2)"""
        base_model = MobileNetV2(
            input_shape=self.input_shape,
            include_top=False,
            weights='imagenet'
        )

        # Freeze base model
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(128, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(self.num_classes, activation='softmax')
        ])

        self.model = model
        return model

    def compile_model(self, learning_rate=0.001):
        """Compile the model"""
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

        self.model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy',
                    keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
        )

        print("\n✓ Model compiled successfully")
        return self.model

    def get_callbacks(self, model_path='models/'):
        """Get training callbacks"""
        os.makedirs(model_path, exist_ok=True)

        callbacks = [
            ModelCheckpoint(
                filepath=os.path.join(model_path, 'best_model.h5'),
                monitor='val_accuracy',
                save_best_only=True,
                verbose=1
            ),
            EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            TensorBoard(
                log_dir=os.path.join('logs', datetime.now().strftime("%Y%m%d-%H%M%S")),
                histogram_freq=1
            )
        ]

        return callbacks

    def train_model(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
        """Train the model"""
        print("\n🚀 Starting model training...")

        callbacks = self.get_callbacks()

        self.history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )

        print("\n✓ Training completed!")
        return self.history

    def plot_training_history(self):
        """Plot training history"""
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        # Accuracy
        axes[0].plot(self.history.history['accuracy'], label='Train Accuracy')
        axes[0].plot(self.history.history['val_accuracy'], label='Val Accuracy')
        axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(alpha=0.3)

        # Loss
        axes[1].plot(self.history.history['loss'], label='Train Loss')
        axes[1].plot(self.history.history['val_loss'], label='Val Loss')
        axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(alpha=0.3)

        plt.tight_layout()
        plt.savefig('visualizations/training_history.png', dpi=300)
        print("✓ Saved training history visualization")

# Initialize and build model
ts_model = TrafficSignModel(input_shape=(64, 64, 3), num_classes=58)
model = ts_model.build_cnn_model()
ts_model.compile_model(learning_rate=0.001)

# Display model architecture
model.summary()



✓ Model compiled successfully


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 4,632,922 (17.67 MB)

 Trainable params: 4,630,490 (17.66 MB)

 Non-trainable params: 2,432 (9.50 KB)

# Model Evaluation

In [17]:
class ModelEvaluator:
    """
    Comprehensive model evaluation
    """

    def __init__(self, model, class_names):
        self.model = model
        self.class_names = class_names

    def evaluate_model(self, X_test, y_test):
        """Evaluate model on test set"""
        print("\n📈 Evaluating model on test set...")

        # Predictions
        y_pred_proba = self.model.predict(X_test)
        y_pred = np.argmax(y_pred_proba, axis=1)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')

        print(f"\n{'='*60}")
        print(f"📊 MODEL EVALUATION RESULTS")
        print(f"{'='*60}")
        print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"Precision: {precision:.4f} ({precision*100:.2f}%)")
        print(f"Recall:    {recall:.4f} ({recall*100:.2f}%)")
        print(f"F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
        print(f"{'='*60}\n")

        # Save metrics
        metrics = {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'f1_score': float(f1),
            'timestamp': datetime.now().isoformat()
        }

        with open('models/metrics.json', 'w') as f:
            json.dump(metrics, f, indent=4)

        return y_pred, y_pred_proba, metrics

    def plot_confusion_matrix(self, y_test, y_pred, figsize=(20, 18)):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_test, y_pred)

        plt.figure(figsize=figsize)
        sns.heatmap(cm, annot=False, fmt='d', cmap='Blues',
                   xticklabels=range(len(self.class_names)),
                   yticklabels=range(len(self.class_names)))
        plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.savefig('visualizations/confusion_matrix.png', dpi=300)
        print("✓ Saved confusion matrix")

    def plot_classification_report(self, y_test, y_pred):
        """Plot detailed classification report"""
        report = classification_report(y_test, y_pred, output_dict=True)

        # Extract metrics for each class
        classes = [k for k in report.keys() if k.isdigit() or k == 'macro avg' or k == 'weighted avg']
        metrics = ['precision', 'recall', 'f1-score']

        data = []
        for cls in classes[:20]:  # Show first 20 classes
            if cls in report:
                data.append([
                    report[cls]['precision'],
                    report[cls]['recall'],
                    report[cls]['f1-score']
                ])

        fig, ax = plt.subplots(figsize=(12, 8))
        x = np.arange(len(classes[:20]))
        width = 0.25

        for i, metric in enumerate(metrics):
            values = [d[i] for d in data]
            ax.bar(x + i*width, values, width, label=metric.capitalize())

        ax.set_xlabel('Class', fontsize=12)
        ax.set_ylabel('Score', fontsize=12)
        ax.set_title('Per-Class Performance Metrics', fontsize=14, fontweight='bold')
        ax.set_xticks(x + width)
        ax.set_xticklabels(classes[:20], rotation=45, ha='right')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)

        plt.tight_layout()
        plt.savefig('visualizations/classification_report.png', dpi=300)
        print("✓ Saved classification report")

    def plot_roc_curves(self, y_test, y_pred_proba, n_classes=10):
        """Plot ROC curves for top classes"""
        # Binarize labels
        lb = LabelBinarizer()
        y_test_bin = lb.fit_transform(y_test)

        fig, ax = plt.subplots(figsize=(12, 8))

        for i in range(min(n_classes, len(self.class_names))):
            fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, lw=2,
                   label=f'Class {i} (AUC = {roc_auc:.2f})')

        ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
        ax.set_xlabel('False Positive Rate', fontsize=12)
        ax.set_ylabel('True Positive Rate', fontsize=12)
        ax.set_title('ROC Curves (Top 10 Classes)', fontsize=14, fontweight='bold')
        ax.legend(loc='lower right', fontsize=9)
        ax.grid(alpha=0.3)

        plt.tight_layout()
        plt.savefig('visualizations/roc_curves.png', dpi=300)
        print("✓ Saved ROC curves")


# Model Persistence

In [21]:
def save_model(model, model_dir='models/'):
    """Save model in multiple formats"""
    os.makedirs(model_dir, exist_ok=True)

    # Save in HDF5 format
    h5_path = os.path.join(model_dir, 'traffic_sign_model.h5')
    model.save(h5_path)
    print(f"✓ Saved model as HDF5: {h5_path}")

    # Save in TensorFlow SavedModel format
    tf_path = os.path.join(model_dir, 'traffic_sign_model_tf')
    model.save(tf_path)
    print(f"✓ Saved model as TensorFlow: {tf_path}")

    # Save model config and weights separately
    config_path = os.path.join(model_dir, 'model_config.json')
    weights_path = os.path.join(model_dir, 'model_weights.h5')

    with open(config_path, 'w') as f:
        f.write(model.to_json())
    model.save_weights(weights_path)

    print(f"✓ Saved model config: {config_path}")
    print(f"✓ Saved model weights: {weights_path}")

save_model(model)


✓ Saved model as HDF5: models/traffic_sign_model.h5


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=models/traffic_sign_model_tf.

# Main Execution Pipeline

In [ ]:
if __name__ == "__main__":

    print("="*70)
    print("TRAFFIC SIGN CLASSIFICATION - ML PIPELINE")
    print("="*70)

    # Create visualization directory
    os.makedirs('visualizations', exist_ok=True)
    os.makedirs('models', exist_ok=True)

    # Note: In practice, you would load actual data here
    # This is a template showing the complete pipeline

    print("\n✓ Pipeline template created successfully!")

    print("\n" + "="*70)

CHINESE TRAFFIC SIGN CLASSIFICATION - ML PIPELINE

✓ Pipeline template created successfully!

